In [2]:
import pickle
import random
import time
import numpy as np

from nltk.translate.bleu_score import corpus_bleu
from tensorflow.python.keras.models import load_model
from tensorflow.python.client import device_lib
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add

In [3]:
curr_folder = "D:/YandexDisk/datasets/"

path_features = curr_folder + "ru-12k-features.pkl"
path_tokenizer = curr_folder + "ru-12k-tokenizer-train.pkl"

path_train_dict = curr_folder + "captions-ru-12k-train.pkl"
path_val_dict = curr_folder + "captions-ru-12k-val.pkl"

In [5]:
def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

def to_lines(data):
    all_vals = list()
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def create_tokenizer(data):
    lines = to_lines(data)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)

    return tokenizer

def find_max_words(data):
    lines = to_lines(data)
    return max(len(l.split()) for l in lines)

### Наборы для обучения и валидации

In [9]:
with open (path_train_dict, 'rb') as f:
    train_dict = pickle.load(f)
print('кол-во подписей .............. %d' % len(train_dict))

with open (path_tokenizer, 'rb') as f:
    tokenizer = pickle.load(f)
vocab_size = len(tokenizer.word_index) + 1
print('размер словаря ............... %d' % vocab_size)

max_words = find_max_words(train_dict)
print('длина предложения в словах ... %d' % max_words)

кол-во подписей .............. 8262
размер словаря ............... 21391
длина предложения в словах ... 22


In [15]:
with open (path_val_dict, 'rb') as f:
    val_dict = pickle.load(f)
val_features = load_image_features(path_features, val_dict)
print('кол-во подписей .............. %d' % len(val_dict))

val_tokenizer = create_tokenizer(val_dict)
val_vocab_size = len(val_tokenizer.word_index) + 1
print('размер словаря ............... %d' % val_vocab_size)

val_max_words = find_max_words(val_dict)
print('длина предложения в словах ... %d' % val_max_words)

кол-во подписей .............. 2360
размер словаря ............... 10450
длина предложения в словах ... 21


# Оценка модели

После того, как модель обучены, мы можем оценить качество её предсказаний в тестовом наборе данных.

Мы оценим модель, создав описания для всех изображений в тестовом наборе данных и оценив эти прогнозы с помощью стандартной функции затрат.

Во-первых, нам нужно иметь возможность сгенерировать описание изображения, используя обученную модель. Это включает в себя передачу маркера начального описания "startseq", генерацию одного слова, а затем рекурсивный вызов модели с сгенерированными словами в качестве входных данных до тех пор, пока не будет достигнут конец маркера последовательности "endseq" или не будет достигнута максимальная длина описания.

Приведённая ниже функция реализует это поведение и генерирует текстовое описание с учётом обученной модели и заданного подготовленного изображения в качестве входных данных. Эта функция вызывает другую функцию map_int_to_word, чтобы отобразить целочисленное предсказание обратно в слово.

In [11]:
def map_int_to_word(integer, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == integer:
            return word

    return None

# 0 seed the generation process
# 1 iterate over the whole length of the sequence
# 2 integer encode input sequence
# 3 pad input
# 4 predict next word
# 5 convert probability to integer
# 6 map integer to word
# 7 stop if we cannot map the word
# 8 append as input for generating the next word
# 9 stop if we predict the end of the sequence
def generate_caption(model, tokenizer, image, max_words):
    in_text = 'startseq'                                    # 0

    for i in range(max_words):                              # 1
        seq = tokenizer.texts_to_sequences([in_text])[0]    # 2
        seq = pad_sequences([seq], maxlen=max_words)        # 3

        y_hat = model.predict([image,seq], verbose=0)       # 4
        y_hat = np.argmax(y_hat)                            # 5

        word = map_int_to_word(y_hat, tokenizer)            # 6
        if word is None:                                    # 7
            break

        in_text += ' ' + word                               # 8

        if word == 'endseq':                                # 9
            break

    return in_text

### BLEU

Мы сгенерируем предсказания для всех изображений в тестовом наборе данных и в наборе данных для обучения.

Приведенная ниже функция с именем evaluate_model() будет оценивать обученную модель по заданному набору подписей изображений и признаков изображений. Фактические и прогнозируемые описания собираются и оцениваются вместе с использованием оценки алгоритма BLEU, который оценивает, насколько сгенерированный текст близок к ожидаемому тексту.

BLEU используется при переводе текста для оценки переведённого текста по одному или нескольким другим переводам.

Here, we compare each generated description against all of the reference descriptions for the photograph. We then calculate BLEU scores for 1, 2, 3 and 4 cumulative n-grams.

The NLTK Python library implements the BLEU score calculation in the corpus_bleu() function. A higher score close to 1.0 is better, a score closer to zero is worse.

In [12]:
def evaluate_model(model, captions, images, tokenizer, max_words):
    actual, predicted = list(), list()

    for key, captions_list in captions.items():
        references = [c.split() for c in captions_list]
        y_hat = generate_caption(model, tokenizer, images[key], max_words)

        actual.append(references)
        predicted.append(y_hat.split())

    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))